# Milerobo!

This notebook demonstrates how to perform Question and Answering over sources.

Once you have scraped enough articles and created a vector store, run the QA on questions and compare ChatGPT's response and your own, while reading what a human expert thinks of ChatGPT.

Source for ChatGPT responses and human expert evaluation: [milelion.com](https://milelion.com/2023/02/11/can-chatgpt-handle-miles-and-points-questions/)

_This notebook and the code in the repository is a toy and is for my education and demonstration purposes only. If you run the code, you are solely liable for running the code and its consequences. The answers provided by the model is provided on an "AS IS" basis only._

In order to use this notebook, you need to
* set your OpenAI key and other variables as an environment variable.
* Run the scraping code and save it in weaviate, a vector db.

## Changes in v2
1. Chunks are now smaller -- from 900 to 300 (ie. 3 time as many)
2. I use GPT-3 to summarise instead of relying on the article.

In [1]:
# Load environment values
%load_ext dotenv
%dotenv

In [3]:
from langchain import VectorDBQA
# Load vector store
import os
import weaviate

resource_owner_config = weaviate.AuthClientPassword(
    username=os.getenv('WEAVIATE_USER'),
    password=os.getenv('WEAVIATE_PASSWORD'),
)

client = weaviate.Client(
    os.getenv('WEAVIATE_URL'),
    auth_client_secret=resource_owner_config,
    additional_headers={
        "X-OpenAI-Api-Key": os.getenv('OPENAI_KEY')
    }
)

from langchain.vectorstores import Weaviate
weaviate = Weaviate(client, "Article", "content")

from langchain.llms import OpenAI

chain = VectorDBQA.from_chain_type(llm=OpenAI(temperature=0, openai_api_key=os.getenv('OPENAI_KEY')), chain_type="stuff", vectorstore=weaviate, return_source_documents=True)


In [5]:
query = "Which Singapore banks pool credit card points?"
chain(query)

{'query': 'Which Singapore banks pool credit card points?',
 'result': ' American Express, Citi, DBS, HSBC, Maybank, OCBC, Standard Chartered, and UOB.',
 'source_documents': [Document(page_content='Credit cards had existed in Singapore for nearly three decades now, and mass adoption had turned them from cherished invitation to common commodity.\xa0\n\nThe ubiquity meant that the vast majority of card perks were underwhelming.\n\nBanks touted complimentary travel insurance, quick card replacement and a maximum liability of S$100 for lost or stolen cards as selling points- hardly things to get the pulse racing.\xa0\n\n\n.MOINSBD_PostFour { width: 320px; height: 100px; }\n@media(min-width: 500px) { .MOINSBD_PostFour { width: 468px; height: 60px; } }\n@media(min-width: 800px) { .MOINSBD_PostFour { width: 728px; height: 90px; } }\nThat all changed when American Express shook up the market with the launch of Premier Tribute in November 1991, which I regard as the first “proper” rewards prog

**Compare ChatGPT**
Q: Which Singapore banks pool credit card points?
A: In Singapore, several banks offer credit card points that can be pooled or transferred to other credit card accounts. Some of the banks that offer this service include:

    CitiBank
    DBS Bank
    OCBC Bank
    UOB Bank

Customers can usually transfer or pool their credit card points through the bank’s online banking portal or mobile app. It’s worth noting that the process of pooling or transferring points may vary by bank, and some banks may have restrictions on the types of accounts that can participate in point pooling. It’s also important to check if there is any expiry date for the points before pooling or transferring them.

**Evaluation**
An answer that’s mostly right, but being mostly right is like being a little bit pregnant.

Citibank, of course, does not pool points across cards, and if we wanted to be even more technical, we would point out that OCBC has three different types of rewards currencies: OCBC$, Travel$ and VOYAGE Miles. These pool within currency (e.g. Travel$ earned on the OCBC 90° N Visa and Mastercard), but not across currency (e.g. Travel $ earned on the OCBC 90° N Visa and OCBC$ earned on the OCBC Titanium Rewards)

In [6]:
query = "Which Singapore credit cards earn non-expiring miles?"
chain(query)

{'query': 'Which Singapore credit cards earn non-expiring miles?',
 'result': ' The American Express Platinum Reserve, Citi Prestige, DBS Vantage, HSBC Visa Infinite, Maybank Visa Infinite, OCBC Voyage, SCB Visa Infinite, and UOB VI Metal Card all earn non-expiring miles.',
 'source_documents': [Document(page_content='Before you dismiss it, however, consider that American Express has the fewest rewards restrictions of any bank in Singapore- you can earn miles on government transactions, education fees and hospital bills, assuming AMEX cards are accepted.\n\nThis makes it a good “card of last resort” when you suspect a particular payment might not earn points with another bank.\n\nPoints Flexibility and ExpiryCardExpiryPartnersXfer FeeMin.\n\nXferAMEX Plat.\n\nReserveNone11None250 milesCiti PrestigeNone11S$26.7510K milesDBS Vantage3 yrs.4S$26.75@10K milesHSBC Visa Infinite37 mo.2S$42.80^10K milesMaybank Visa InfiniteNone4None5K milesOCBC VOYAGENone1None1 mileSCB\n\nVisa InfiniteNone10S$

Q: Which Singapore credit cards earn non-expiring miles?

A: In Singapore, some credit cards offer non-expiring miles, meaning that the miles earned through credit card spending do not have an expiration date. Here are some examples of credit cards that offer non-expiring miles:

    Citi PremierMiles Visa Card: Miles earned with this card do not expire as long as the card is active.
    UOB PRVI Miles Visa Card: Miles earned with this card do not expire as long as the account is in good standing
    DBS Altitude Visa Card: Miles earned with this card do not expire as long as the card account is in good standing
    OCBC 90°N Card: Miles earned with this card do not expire as long as the account is in good standing.

Please note that the terms and conditions of the non-expiring miles program may vary, such as the minimum spend required to earn miles. the transfer ratio of miles to frequent flyer programs, and the expiration date of the card. Its always best to check the bank’s website or contact the bank’s customer service for more information on the credit card’s program.

–ChatGPT

It’s a bit strange that ChatGPT defaults to mentioning the Visa versions of (1), (2) and (3), when they come in other flavours as well.

But anyway, like the previous question, ChatGPT is partially correct. The Citi PremierMiles Card, DBS Altitude Card and OCBC 90°N Card have non-expiring miles, but the UNI$ earned on the UOB PRVI Miles Card expire after 2 years.

In [8]:
query = " Which Singapore bank has the most points transfer partners?"
chain(query)

{'query': ' Which Singapore bank has the most points transfer partners?',
 'result': ' Citibank has the most points transfer partners in Singapore, with 12 airline and hotel programmes to choose from.',
 'source_documents': [Document(page_content='Which bank has the most transfer partners?While the addition of AirAsia brings UOB’s transfer partner stable to three, it’s still a far cry from the big boys in the market.\xa0\n\nClick to enlargeThe leader by far remains Citibank with its 10 airline and one hotel partner, followed by American Express with nine airline and two hotel partners, and Standard Chartered with eight airline and two hotel partners (no, they’ve not removed KrisFlyer).\xa0\n\nConclusionUOB cardholders now have three airline transfer partners to convert their UNI$ to, but I just can’t see any scenario in which you’d opt for AirAsia.\n\nEven if you need airasia tickets, you’re better off buying them on sale than turning your hard-earned UNI$ into airasia points.\xa0\n\n\

Q: Which Singapore bank has the most points transfer partners?

A: In Singapore, banks have partnerships with different frequent flyer programs and loyalty schemes, which allows customers to transfer their credit card points to these programs. The bank with the most points transfer partners may vary over time, but currently, DBS and UOB have a wide variety of transfer partners.

DBS has partnerships with frequent flyer programs such as Singapore Airlines KrisFlyer, Cathay Pacific Asia Miles, and Thai Airways Royal Orchid Plus, as well as hotel loyalty programs like Marriott Bonvoy, Hilton Honors, and Accor Live Limitless.

UOB also has partnerships with frequent flyer programs such as Singapore Airlines KrisFlyer, Cathay Pacific Asia Miles, and Thai Airways Royal Orchid Plus, as well as hotel loyalty programs like Marriott Bonvoy, Hilton Honors, and Accor Live Limitless, plus other loyalty schemes like Grab Rewards.

Citibank also has a good number of transfer partners such as Singapore Airlines KrisFlyer, Cathay Pacific Asia Miles, Qantas Frequent Flyer and Thai Airways Royal Orchid Plus.

It’s always best to compare the transfer partners of different banks and credit cards to see which program aligns best with your travel and redemption goals. It’s also important to keep in mind that some banks may charge a fee for transferring points and some partners may have different transfer ratios.

–ChatGPT

DBS and UOB most certainly do not have the most transfer partners in Singapore, with the former having four and the latter having three (AirAsia Rewards was added recently, in case you’re out of the loop). And they both don’t partner with THAI, Marriott Bonvoy, Hilton Honors or Accor Live Limitless.

Citibank is a right call, although the absence of American Express and Standard Chartered means this answer isn’t anywhere near adequate.

In [9]:
query = "How do I transfer DBS Points to KrisFlyer?"
chain(query)

{'query': 'How do I transfer DBS Points to KrisFlyer?',
 'result': ' You can transfer DBS Points to KrisFlyer via the Kris+ transfer portal. The transfer rate is 1 DBS Point to 1.7 miles, and there is no transfer fee. The transfer is instant.',
 'source_documents': [Document(page_content='DBS PointsRead More1 UNI$: 1.7 miles1,000 UNI$26\n\nSTAR$: 1 mile9,880 STAR$KrisPay miles can then be transferred immediately to KrisFlyer miles at a 1:1 ratio.\xa0\n\nIf you’re transferring bank points, do note that the Kris+ transfer rate represents a 15% haircut compared to the number of miles you’d receive if you used the bank’s transfer portal.\n\n(e.g. 100 DBS Points is normally worth 200 miles via the DBS Rewards portal, but only 170 miles via Kris+).Transfer DBS Points via…\xa0\xa0Kris+\xa0DBS Rewards PortalMin. Transfer100\n\nDBS Points5,000 DBS PointsTransfer FeeNoneS$26.75Transfer TimeInstant2-3 working daysTransfer Ratio1 DBS Point = 1.7 miles1\n\nDBS Point = 2 milesOn the plus side, there

Q: How do I transfer DBS Points to KrisFlyer?

A: DBS points can be transferred to Singapore Airlines KrisFlyer miles through the DBS Rewards program. The transfer process is generally simple and can be done online. Here’s an outline of the steps:

  *  Log in to your DBS Rewards account online.

  * Under the “Redeem” tab, select the “Transfer to Miles” option.

  *  Choose Singapore Airlines KrisFlyer as the transfer partner.

  *  Enter the number of DBS points you wish to transfer and confirm the transfer.

  *  The KrisFlyer miles will be credited to your KrisFlyer account within 3 working days.

It’s worth noting that DBS points can be transferred to KrisFlyer miles at a 1:1 ratio, and there’s no minimum transfer amount but a minimum of 2,000 DBS Points need to be converted and there may be a transfer fee. Make sure to check the terms and conditions of the DBS Rewards program to ensure that you understand any applicable fees, expiration dates or other restrictions.

-ChatGPT

The steps are generally correct, if somewhat poorly labelled (you look under the “Travel” tab for the KrisFlyer mile option). That said, I’m surprised ChatGPT gave the correct crediting timeline of three working days, instead of DBS’s official but excessively CYA timeframe of 1-2 weeks.

You should probably ignore everything after step (5), since DBS Points transfer to KrisFlyer miles at a 1:2 ratio, with a minimum block of 5,000 DBS Points.

In [10]:
query = "How do I link my KrisFlyer and Shangri-La Circle accounts?"
chain(query)

{'query': 'How do I link my KrisFlyer and Shangri-La Circle accounts?',
 'result': " You can link your KrisFlyer and Shangri-La Circle accounts by signing in to your KrisFlyer account on KrisShop.com. You'll see a pop-up window appear automatically, and you should make sure to check the box in red at the time of registration. If you accidentally close this pop-up or don’t see it, you can also login to your KrisFlyer account on singaporeair.com, click on Miles > How to Earn, locate the KrisShop section and click on Link Account. A confirmation email will be sent once your accounts have been linked.",
 'source_documents': [Document(page_content='You’ll see the following pop-up window appear automatically.\xa0\n\nMake sure to check the box in red at the time of registrationIf you accidentally close this pop-up or don’t see it, you can also login to your KrisFlyer account on singaporeair.com, click on Miles > How to Earn, locate the KrisShop section and click on Link Account.\n\nA confirma

Q: How do I link my KrisFlyer and Shangri-La Circle accounts?

A: To link your KrisFlyer and Shangri-La Circle accounts, you will need to follow these steps:

    * Log in to your KrisFlyer account on the KrisFlyer website.

    * Go to the “Link Partners” section of the website and select “Shangri-La Circle” from the list of partners.

    * Follow the instructions on the screen to link your accounts. This may involve entering your Shangri-La Circle account information or logging in to your Shangri-La Circle account through the KrisFlyer website.

    * Once your accounts are linked, you will be able to earn KrisFlyer miles when you make reservations at Shangri-La Hotels and Resorts. You will also be able to redeem your miles for stays at Shangri-La Hotels and Resorts.

Please note that the specific steps may vary slightly depending on your location and the version of the website. The partnership between KrisFlyer and Shangri-La Circle is also subject to change and might no longer exist, it is best to check the official website for the most up-to-date information.

-ChatGPT

The general sequence is correct, although people are likely to get lost at step (2) because there’s no such section on the KrisFlyer website called “link partners”. You’ll need to go to Profile > Partner Programmes to find the option to add Shangri-La.

In [11]:
query = "How do I find the MCC of a merchant before paying?"
chain(query)

{'query': 'How do I find the MCC of a merchant before paying?',
 'result': ' You can use DBS Payment Controls and the DBS digibot to check the MCC of a merchant before paying. You can temporarily block a DBS card via the DBS digibank app, then go to the merchant and swipe your locked card (or enter your details online, as the case may be). The transaction will fail since your card is locked, but you can then use the DBS digibot to check the MCC. Once you’ve confirmed the MCC, you can use whichever card you think is best.',
 'source_documents': [Document(page_content='But there’s another way.\n\nChecking MCCs through DBSDoes SKIRT code as a restaurant or hotel?\n\nNow you’ll knowI previously wrote about how the DBS digibot can be used to check MCCs for transactions charged to DBS cards.\xa0\n\nHowever, you can only do this after the fact.\n\nIn other words, you have to spend first, then make an enquiry about the transaction.\n\nOr do you?\n\nWhat if I told you there’s a way to confirm M

Q: How do I find the MCC of a merchant before paying?

A: Merchant Category Code (MCC) is a four-digit number assigned to a merchant by the credit card issuer. The MCC is used to classify the type of business or service that the merchant provides.

There are a few ways to find the MCC of a merchant before paying:

   * Contact the merchant directly: You can ask the merchant what their MCC is. Some merchants may have it displayed on their website, or they may be able to provide it to you over the phone or in person
   *  Check your credit card statement: If you’ve made a purchase from the merchant before, the MCC should be listed on your credit card statement
   * Search online: You can search for the merchant’s name along with the term “MCC” to see if the information is available online
   * Contact your credit card issuer: Some credit card issuers may have a database of MCCs for merchants, and they may be able to provide you with the MCC for a specific merchant.

Please note that not all merchants have unique MCCs, it may depend on the merchant’s type of business and the way they process the transactions.

-ChatGPT

Ah, the age old “what’s the MCC” question.

It would have been amazing if ChatGPT had managed to suss out my article on how you can check MCCs before spending via DBS cards, but its answers were reasonable enough, if a little commonsensical.

ChatGPT even raised a good point about how not all merchants would have a single MCC, e.g. retailers like Amazon and Lazada which sell a range of products. OCBC Titanium Rewards, you may remember, is planning to exclude grocery transactions processed under Amazon from 1 March 2023 while leaving other Amazon transactions untouched.

In [13]:
query = "Should I use the DBS Woman’s World Card or Citi Rewards Card for online spending?"
chain(query)

{'query': 'Should I use the DBS Woman’s World Card or Citi Rewards Card for online spending?',
 'result': " It depends on your individual needs and preferences. The DBS Woman's World Card offers 4 mpd on all online spending, capped at S$2,000 per month, and points are valid for one year. The Citi Rewards Card offers 4 mpd on online spending, with a longer points expiry period of up to five years, but the points are not pooled with other DBS cards.",
 'source_documents': [Document(page_content='Otherwise, you’ll have access to the usual generic DBS benefits, which can be found here.\xa0\n\nSummary Review: DBS Woman’s World CardApply HereGiven the sheer number of online transactions performed on a daily basis, you absolutely need to have a dedicated online spending card.\n\nWith an earn rate of 4 mpd, the DBS Woman’s World Card is one of the best options for this category.\n\nThe monthly 4 mpd cap of S$2,000 is also higher than other cards which offer bonuses on online spending, like the

Q: Should I use the DBS Woman’s World Card or Citi Rewards Card for online spending?

A: Both the DBS Woman’s World Card and the Citi Rewards Card have rewards programs that can be beneficial for online spending. However, the best card for you will depend on your specific spending habits and preferences.

The DBS Woman’s World Card gives you 6X rewards on online fashion and beauty, 4X rewards on online travel bookings and 2X rewards on online shopping and everyday spends.

The Citi Rewards Card gives you 10X rewards points on online shopping and dining.

You should consider your spending habits and which category you spend more on, then compare the rewards, benefits, and fees of each card to determine which one will give you the most value. It’s also worth considering any promotions or bonus rewards that the card issuer may be offering.

-Chat GPT

Swing and a miss for ChatGPT here. It spouts gibberish about the DBS Woman’s World Card’s 2X, 4X and 6X rewards, when it’s as simple as 10X rewards for all online spending that’s not on the general exclusions list.

Likewise, it limits the 10X rewards points on the Citi Rewards Card to online shopping and dining, when it should be all online transactions except travel and mobile wallet.

I would have loved to see ChatGPT bring in other factors like points expiry, number of transfer partners, and points pooling when comparing these two cards.

In [14]:
query = "Should I use the UOB Preferred Platinum Visa or UOB Visa Signature for contactless spending?"
result = chain(query)

result['result']

' The UOB Visa Signature is the better option for contactless spending, as it has a higher monthly 4 mpd cap.'

Q: Should I use the UOB Preferred Platinum Visa or UOB Visa Signature for contactless spending?

A: Whether you should use the UOB Preferred Platinum Visa or the UOB Visa Signature for contactless spending will depend on your individual spending habits and preferences. Both cards offer rewards for contactless spending, but they have different earning rates and benefits.

The UOB Preferred Platinum Visa offers 5% cashback on contactless transactions, and UOB One card allows customers to pool rewards across multiple UOB credit cards and enjoy higher cashback rate on their spending. It also offers other benefits such as travel insurance, discounts on hotel stays and dining.

The UOB Visa Signature offers 3% cashback on contactless transactions, and also offers a wide range of benefits such as travel insurance, discounts on hotel stays and dining, and access to exclusive events.

It’s important to compare the earning rates, rewards, benefits and fees of each card before making a decision. It’s also recommended to look at your spending habits and see which card aligns best with your spending patterns and preferences. You can also check the terms and conditions of the card to see which one offers the best value for you.

-ChatGPT

5% cashback on contactless transactions ? That doesn’t compute, somehow. Even if you took the UNI$ earned on the UOB Preferred Platinum Visa and converted them into cash rebates (don’t!), a 4 mpd earn rate translates into a 2% rebate, not 5%. Ditto the UOB Visa Signature; I have no idea how they got 3% cashback on contactless spend.

I was really hoping ChatGPT would be able to highlight important differences like the minimum spend requirement of the UOB Visa Signature, its higher income requirement (not that it has any practical impact, but at least on paper…), its higher 4 mpd cap, or the “mobile payments only” restriction of the UOB Preferred Platinum Visa.

In [15]:
# Higher temperature for opinions

chain = VectorDBQA.from_chain_type(llm=OpenAI(temperature=0.5, openai_api_key=os.getenv('OPENAI_KEY')), chain_type="stuff", vectorstore=weaviate)


In [16]:
query = "Which $120K credit card in Singapore is the best?"
chain.run(query)

' The best $120K credit card in Singapore depends on your lifestyle and travel patterns. The Citi Prestige and HSBC Visa Infinite offer unlimited lounge access and airport limo benefits, plus relatively generous travel insurance coverage, making them well-suited to frequent flyers. The DBS Vantage is a good option for miles accumulators, as it has the fewest rewards restrictions of any bank in Singapore.'

Q: Which $120K credit card in Singapore is the best?

A: In Singapore, there are several credit cards that offer high credit limits, including those with limits of $120,000. The best credit card for you will depend on your individual spending habits and preferences. Here are some popular options:

   *  Citi Prestige Card: This card offers a high credit limit of $120,000, as well as a wide range of benefits and rewards, including travel insurance, airport lounge access, and complimentary hotel nights
   *  UOB Visa Infinite: This card offers a high credit limit of $120,000. as well as a wide range of benefits and rewards, including travel insurance, airport lounge access, and complimentary hotel nights.
   *  DBS Insignia Visa Infinite Card: This card offers a high credit limit of $120,000, as well as a wide range of benefits and rewards, including travel insurance, airport lounge access, and complimentary hotel nights.
   *  Standard Chartered Visa Infinite: This card offers a high credit limit of $120,000, as well as a wide range of benefits and rewards, including travel insurance. airport lounge access, and complimentary hotel nights.

These credit cards have high annual fees, and also high interest rates, and they may have different rewards and benefits, it is important to compare the terms and conditions of each card, including the interest rate, annual fee, rewards, and benefits, before applying to select the one that best fits your needs.

First of all, credit to ChatGPT for being able to understand the concept of a $120K credit card. I didn’t say “credit card with a $120,000 income requirement”, but it was still able to select credit cards that fall into this segment- even two which technically have income requirements of S$150,000 (UOB Visa Infinite, SC Visa Infinite), but are still considered part of the $120K gang! It did mix up the DBS Insignia (S$500,000 income requirement) with the DBS Vantage, however.

That said, ChatGPT confuses the concepts of credit limit and income requirement, and the analysis of benefits are rudimentary at best. All four cards have the same boilerplate “wide range of benefits and rewards, including travel insurance, airport lounge access, and complimentary hotel nights”.

In [17]:
query = "What is the best miles credit card for groceries in Singapore?"
chain.run(query)

' The HSBC Revolution card is a good option for groceries as it earns 4 mpd with a maximum spend of S$1,000 per calendar month.'

Q: What is the best miles credit card for groceries in Singapore?
A: There are several credit cards in Singapore that offer rewards or miles for grocery purchases, but the best one for you will depend on your individual spending habits and preferences. Some popular options include:

   *  Citi PremierMiles Visa Card: This card offers up to 2 miles per S$1 spent on everyday grocery purchases at supermarkets, and 10,000 miles as welcome bonus
   *   UOB PRVI Miles Visa Card: This card offers up to 3.3 miles per S$1 spent on online grocery shopping and overseas online shopping, and 8,000 miles as welcome bonus
   *  OCBC 90°N Card: This card offers up to 3 miles per S$1 spent on online grocery and food delivery, and 6,000 miles as welcome bonus

It is always recommended to compare the benefits, rewards, fees and other features of credit card before applying.

-ChatGPT

ChatGPT completely tanked this answer. None of the above is in any way accurate, and I can’t even think of limited-time promotions where the above was true, making me wonder where it sourced this information from.

As I’m sure you know, the HSBC Revolution or UOB Lady’s Card (with family set as the 10X category) are where you should turn for this.

In [18]:
query = "What is the best miles credit card for petrol in Singapore?"
chain.run(query)

' The best miles credit card for petrol in Singapore is UOB Visa Signature or UOB Preferred Platinum Visa.'

Q: What is the best miles credit card for petrol in Singapore?

A: There are several credit cards in Singapore that offer rewards or miles for petrol purchases, but the best one for you will depend on your individual spending habits and preferences. Some popular options include:

   * Citi Cash Back Card: This card offers 8% cashback on petrol spend with a cap of $25 per month
   * UOB One Card: This card offers 8% rebate on petrol spend at Esso and Shell stations, with a cap of $70 per statement cycle
   * Standard Chartered Unlimited Cashback Card: This card offers 5% cashback on all purchases, including petrol
   * OCBC 365 Credit Card: This card offers 6% cashback on grocery and petrol spend with a cap of $80 per statement cycle.

It is always recommended to compare the benefits, rewards, fees and other features of credit card before applying.

-ChatGPT

Where the heck did this answer come from? Leaving aside the fact it offered me four cashback options when I asked for miles, the cashback options it presents aren’t even accurate.

The Standard Chartered Unlimited Cashback Card (now known as the Simply Cash Card, fyi) offers a flat 1.5% cashback on all spend, the OCBC 365 Credit Card earns 5% cashback on petrol, and the UOB One gets additional rebates at Shell and SPC, not Esso.

In [19]:
query = "What is the best miles credit card for insurance in Singapore?"
chain.run(query)

' The best miles credit card for insurance in Singapore is the Maybank Visa Infinite or Maybank Horizon Visa Signature, which earns 0.29-0.6 mpd on insurance transactions.'

Q: What is the best miles credit card for insurance in Singapore?

A: There are several miles credit cards in Singapore that offer rewards for insurance premium payments, but the best one for you may depend on your specific needs and spending habits. Some popular options include:

   * Citi PremierMiles Visa Card: This card offers 1.2 miles for every $1 spent on insurance premium payments, and 2 miles for every $1 spent on air tickets, hotels, and other travel expenses
   * American Express KrisFlyer Ascend Credit Card: This card offers 1.2 miles for every $1 spent on insurance premium payments, and 1.5 miles for every $1 spent on online travel bookings
   * UOB PRVI Miles Visa Card: This card offers 1.4 miles for every $1 spent on insurance premium payments, and 2 miles for every $1 spent on air tickets booked through the UOB Travel portal

It’s important to note that these are just a few examples and there are many other credit card options in the market. It is best to compare the benefits and rewards of the different cards and choose the one that best aligns with your spending habits and travel preferences.

-ChatGPT

The only one here that’s remotely correct is the AMEX KrisFlyer Ascend, and even then, only on insurance products purchased through American Express authorised channels. Everything else is flat out wrong.

In [20]:
query = "What are the best redemption sweet spots in the Singapore Airlines KrisFlyer programme?"
chain.run(query)

' The best redemption sweet spots in the Singapore Airlines KrisFlyer programme include short-haul advance upgrades, premium economy advance upgrades, 1,500 KrisPay miles, 25% bonus KrisFlyer miles, 50% bonus KrisFlyer miles, 5,000 KrisFlyer miles redemption discount, double KrisFlyer miles, 1,000 Elite miles, and KrisFlyer Experiences.'

Q: What are the best redemption sweet spots in the Singapore Airlines KrisFlyer programme?
A: The Singapore Airlines KrisFlyer program is a frequent flyer program that allows members to earn miles for flights with Singapore Airlines, SilkAir, and Star Alliance partners. These miles can be redeemed for award flights, upgrades, and other travel-related items.

Some sweet spots in the KrisFlyer program for award redemptions include:

   * Short-haul flights within Southeast Asia: These flights can be booked for as few as 7,500 miles one-way in economy class.

   * Flights to Japan, South Korea, and Australia: These flights can be booked for 20,000-25,000 miles one-way in economy class.

   * Flights to the US: These flights can be booked for 35,000-45,000 miles one-way in economy class.

   * Flights to Europe: These flights can be booked for 45,000-57,500 miles one-way in economy class.

   * Upgrade to Business Class: KrisFlyer members can also use their miles to upgrade from economy to business class on Singapore Airlines and SilkAir flights. The miles required for an upgrade varies depending on the route, but it can be a good value.

It’s worth noting that award availability and the cost in miles can vary depending on the route, time of year, and booking window, so it’s always best to check the availability and costs before redeeming your miles.

Some of the information here is out of date- SilkAir no longer exists, the cheapest one-way Economy awards through KrisFlyer now start from 8,500 miles ever since the July 2022 devaluation, and the other mileage figures it quotes are off by a few thousand miles. I also disagree that buying an Economy Class ticket and using miles to upgrade to Business Class is good value, unless someone else is paying for the ticket.

But in a broader sense, is ChatGPT correct to say that Southeast Asia, Japan, Australia, the USA and Europe are sweet spots for KrisFlyer?

That’s probably a bit optimistic, since no programme has that many sweet spots.

   * For short-haul trips within Southeast Asia, I’d go with British Airways Executive Club and their good value short-haul awards. KrisFlyer isn’t terrible, especially if you can find a destination during Spontaneous Escapes, but it wouldn’t necessarily be the first thing that comes to mind
   * For flights to Japan, I’d go with Alaska Mileage Plan and their Japan Airlines awards at 25,000 miles in Business Class (still good value, for now!)
   * For flights to Europe, I’d go with Turkish Miles&Smiles, since Singapore to Europe is just 45,000 miles in Business Class. I know it’s hard to find award space, and I know the fuel surcharges are high, but it’s still less than half the miles KrisFlyer wants
   * For flights to the USA, I’d go with Alaska Mileage Plan and their Cathay Pacific awards at 50,000 miles in Business Class

If anything, I was hoping ChatGPT would pick up the Singapore to Middle East/Africa sweet spot, since 56,500 miles for a one-way Business Class ticket from Singapore to Cape Town (featuring almost 14 hours of flight time) is probably one of KrisFlyer’s best redemptions.